# Import Packages

In [2]:
import kagglehub
import sqlite3
import os
import pandas as pd
import numpy as np
from fuzzywuzzy import process

import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# Load Data

## Soccer Data

In [5]:
# Download the dataset from Kaggle and return the path where it's stored
path = kagglehub.dataset_download("hugomathien/soccer")

# Connect to the SQLite database file located in the downloaded dataset
conn = sqlite3.connect(os.path.join(path, 'database.sqlite'))

# Retrieve tables
sqlite_sequence = pd.read_sql_query("SELECT * FROM sqlite_sequence", conn)
matches_df = pd.read_sql_query("SELECT * FROM Match", conn)
player_df = pd.read_sql_query("SELECT * FROM Player", conn)
player_attributes_df = pd.read_sql_query("SELECT * FROM Player_Attributes", conn)
league_df = pd.read_sql_query("SELECT * FROM League", conn)
country_df = pd.read_sql_query("SELECT * FROM Country", conn)
team_df = pd.read_sql_query("SELECT * FROM Team", conn)
team_attributes_df = pd.read_sql_query("SELECT * FROM Team_Attributes", conn)

In [6]:
# Display
matches_df.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


In [7]:
# Display
player_df.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


In [8]:
# Display
player_attributes_df.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [9]:
# Display
league_df.head()

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A


In [10]:
# Display
country_df.head()

,id,name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy


In [11]:
# Display
team_df.head()

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB


In [12]:
# Display
team_attributes_df.head()

,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,1,434,9930,2010-02-22 00:00:00,60,Balanced,NaN,Little,50,Mixed,...,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
1,2,434,9930,2014-09-19 00:00:00,52,Balanced,48.0,Normal,56,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
2,3,434,9930,2015-09-10 00:00:00,47,Balanced,41.0,Normal,54,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
3,4,77,8485,2010-02-22 00:00:00,70,Fast,NaN,Little,70,Long,...,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover
4,5,77,8485,2011-02-22 00:00:00,47,Balanced,NaN,Little,52,Mixed,...,52,Normal,Organised,47,Medium,47,Press,52,Normal,Cover


## Expenditure Data

In [14]:
# Define the base directory containing the league folders
base_dir = "data/expenditure"

# List of leagues to load data from
leagues = ["bundesliga", "laliga", "ligue-1", "premier-league", "primeira-liga", "serie-a"]

# Initialize an empty list to collect data from each file
all_data = []

# Loop through each league directory to load and append seasonal expenditure data
for league in leagues:
    # Construct the full path to the league's directory
    league_dir = os.path.join(base_dir, league)

    # Loop through each season's CSV file in the league directory
    for file in os.listdir(league_dir):
        # Check if the file is a CSV
        if file.endswith(".csv"):
            # Define the file path and load the CSV into a DataFrame
            file_path = os.path.join(league_dir, file)
            df = pd.read_csv(file_path)

            # Extract the season year from the file name (e.g., "2008" from "2008.csv")
            season = file.split(".")[0]
            # Add the `season` and `league` columns to the DataFrame
            df['season'] = f"{season}/{int(season) + 1}"

            # Append the DataFrame to the list of all data
            all_data.append(df)

# Concatenate all league data into a single DataFrame
expenditure_df = pd.concat(all_data, ignore_index=True)

# Display
expenditure_df.head()

,Unnamed: 0,Club,Competition,Expenditure,Arrivals,Income,Departures,Balance,season
0,0,Hamburger SV,Bundesliga,€34.45m,15,€45.20m,14,€10.75m,2008/2009
1,1,VfL Wolfsburg,Bundesliga,€33.10m,19,€4.53m,22,€-28.57m,2008/2009
2,2,FC Schalke 04,Bundesliga,€15.75m,8,€9.00m,11,€-6.75m,2008/2009
3,3,Borussia Dortmund,Bundesliga,€13.75m,13,€12.30m,17,€-1.45m,2008/2009
4,4,TSG 1899 Hoffenheim,Bundesliga,€11.95m,10,€125k,16,€-11.83m,2008/2009


# Prepare Data for Merging

## Clean Expenditure Data

In [17]:
# Remove unnecessary columns and rename for consistency
expenditure_df.drop(columns=['Unnamed: 0'], inplace=True, errors='ignore')  # Safeguard for possible absence of 'Unnamed: 0'
expenditure_df.rename(columns={"Competition": "league"}, inplace=True)

# Function to convert currency strings to float
def convert_currency(value):
    """
    Converts a currency string to a float. Interprets:
    - "m" as million euros
    - "k" as thousand euros
    - "-" or "+-0" as 0.0
    """
    # Handle special cases for zero values
    if value in ['-', '+-0']:
        return 0.0
    # Remove the euro symbol for processing
    value = value.replace('€', '')
    # Convert millions
    if 'm' in value:
        return float(value.replace('m', '')) * 1e6
    # Convert thousands
    elif 'k' in value:
        return float(value.replace('k', '')) * 1e3
    # Convert directly if no unit suffix
    return float(value)

# Apply the currency conversion function to relevant columns
expenditure_df['Expenditure'] = expenditure_df['Expenditure'].apply(convert_currency)
expenditure_df['Income'] = expenditure_df['Income'].apply(convert_currency)
expenditure_df['Balance'] = expenditure_df['Balance'].apply(convert_currency)

# Exclude data for the 2016/2017 season since we do not have any data for it in matches_df
expenditure_df = expenditure_df[expenditure_df['season'] != '2016/2017']

# Display to confirm changes
expenditure_df.head()

,Club,league,Expenditure,Arrivals,Income,Departures,Balance,season
0,Hamburger SV,Bundesliga,34450000.0,15,45200000.0,14,10750000.0,2008/2009
1,VfL Wolfsburg,Bundesliga,33100000.0,19,4530000.0,22,-28570000.0,2008/2009
2,FC Schalke 04,Bundesliga,15750000.0,8,9000000.0,11,-6750000.0,2008/2009
3,Borussia Dortmund,Bundesliga,13750000.0,13,12300000.0,17,-1450000.0,2008/2009
4,TSG 1899 Hoffenheim,Bundesliga,11950000.0,10,125000.0,16,-11830000.0,2008/2009


## Add columns to expenditure data for merging

In [19]:
# add league_id column
def leaguename_to_id(league_name):
    league_dict = {
        "Bundesliga": 7809,
        "LaLiga": 21518,
        "Ligue 1": 4769,
        "Premier League": 1729,
        "Liga Portugal": 17642,
        "Serie A": 10257}
    return league_dict.get(league_name, None)
expenditure_df['league_id'] = expenditure_df['league'].apply(leaguename_to_id)


# Mapping team names in team_df to their team_api_id
team_name_to_api_id = team_df.set_index('team_long_name')['team_api_id'].to_dict()

# List of all team names for fuzzy matching reference
team_names = list(team_name_to_api_id.keys())

# Standardize team names based on known discrepancies
def standardize_team_name(name):
    # Dictionary for known discrepancies in team naming
    name_replacements = {
        "Reggina Calcio": "reggio calabria",  # Treat "Reggina" as "Reggio Calabria" for consistency
        "ESTAC Troyes": "ES Troyes AC"
    }
    return name_replacements.get(name, name)  # Replace name if found in replacements, otherwise return as is

# Function to retrieve team_api_id with manual mapping, direct lookup, and fuzzy matching
def get_team_api_id(name, threshold=80):
    """
    Returns the team_api_id for a given team name.
    Attempts direct lookup first, followed by fuzzy matching if needed.

    Parameters:
        name (str): The team name to lookup.
        threshold (int): Minimum fuzzy match score to accept.

    Returns:
        int or None: Returns team_api_id if match found, else None.
    """
    # Attempt direct lookup
    team_api_id = team_name_to_api_id.get(name)
    if team_api_id:
        return team_api_id

    # If direct match fails, use fuzzy matching
    match, score = process.extractOne(name, team_names)
    if score >= threshold:
        return team_name_to_api_id[match]
    else:
        print(f"No match found for team name: {name}")
        return None

# Apply the standardization function to clean names in `expenditure_df`
expenditure_df['Club'] = expenditure_df['Club'].apply(standardize_team_name)

# Map team_api_id to `expenditure_df` based on standardized and matched names
expenditure_df['team_api_id'] = expenditure_df['Club'].apply(get_team_api_id)

# Display the updated DataFrame to confirm changes
print(expenditure_df.head())

# Check for any remaining unmatched team names
unmatched_teams = expenditure_df[expenditure_df['team_api_id'].isnull()]['Club'].unique()
print("Remaining unmatched teams:", unmatched_teams)

                  Club      league  Expenditure  Arrivals      Income  \
0         Hamburger SV  Bundesliga   34450000.0        15  45200000.0   
1        VfL Wolfsburg  Bundesliga   33100000.0        19   4530000.0   
2        FC Schalke 04  Bundesliga   15750000.0         8   9000000.0   
3    Borussia Dortmund  Bundesliga   13750000.0        13  12300000.0   
4  TSG 1899 Hoffenheim  Bundesliga   11950000.0        10    125000.0   

   Departures     Balance     season  league_id  team_api_id  
0          14  10750000.0  2008/2009       7809         9790  
1          22 -28570000.0  2008/2009       7809         8721  
2          11  -6750000.0  2008/2009       7809        10189  
3          17  -1450000.0  2008/2009       7809         9789  
4          16 -11830000.0  2008/2009       7809         8226  
Remaining unmatched teams: []


## Save expenditure data to CSV

In [54]:
# Create the directory
output_dir = "Data"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the expenditure_df DataFrame to a CSV file
output_path = os.path.join(output_dir, "expenditure.csv")
expenditure_df.to_csv(output_path, index=False)

## Remove duplicate rows in soccer data

In [21]:
# List of all dataframes to process
dataframes = {
    "matches_df": matches_df,
    "player_df": player_df,
    "Player_Attributes_df": player_attributes_df,
    "League_df": league_df,
    "Country_df": country_df,
    "Team_df": team_df,
    "Team_Attributes_df": team_attributes_df
}

# Check and remove duplicates for each DataFrame
for name, df in dataframes.items():
    duplicate_rows = df[df.duplicated()]
    print(f"Number of duplicate rows in {name}: {duplicate_rows.shape[0]}")

Number of duplicate rows in matches_df: 0
Number of duplicate rows in player_df: 0
Number of duplicate rows in Player_Attributes_df: 0
Number of duplicate rows in League_df: 0
Number of duplicate rows in Country_df: 0
Number of duplicate rows in Team_df: 0
Number of duplicate rows in Team_Attributes_df: 0


## Convert Date Columns to DateTime Format

In [23]:
for name, df in dataframes.items():
    print(f"Converting date columns in {name}...")
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'], errors='coerce')

Converting date columns in matches_df...
Converting date columns in player_df...
Converting date columns in Player_Attributes_df...
Converting date columns in League_df...
Converting date columns in Country_df...
Converting date columns in Team_df...
Converting date columns in Team_Attributes_df...


## Filter matches_df to the 6 leagues with expenditure data

In [25]:
# League IDs (England Premier League, France Ligue 1, Germany 1. Bundesliga,
# Italy Serie A, Portugal Liga ZON Sagres, Spain LIGA BBVA)
league_ids = [1729, 4769, 7809, 10257, 17642, 21518]

# Filter the DataFrame to only include rows with these league IDs and reset the index
matches_df = matches_df[matches_df['league_id'].isin(league_ids)].reset_index(drop=True)

# Merge matches_df and expenditure_df

In [27]:
def get_team_expenditure_data(team_api_id, season):
    """
    Get the expenditure data for a team in a given season.

    Parameters:
    team_api_id (int): The team's API ID.
    season (str): The season in the format 'YYYY/YYYY'.

    Returns:
    dict: A dictionary containing the expenditure data for the team in the season.
    """
    # Filter the `expenditure_df` DataFrame for the specified team and season
    team_data = expenditure_df[(expenditure_df['team_api_id'] == team_api_id) & (expenditure_df['season'] == season)][['Expenditure']]

    # If no data is found, return a dictionary with zero values
    if team_data.empty:
        return {'Expenditure': 0} 

    # Return the expenditure data as a dictionary
    return team_data.iloc[0].to_dict()

# Apply the function to get home and away team expenditures and other columns
for row in matches_df.itertuples():
    home_team_id = row.home_team_api_id
    away_team_id = row.away_team_api_id
    season = row.season

    # Get expenditure data for home and away teams
    home_team_data = get_team_expenditure_data(home_team_id, season)
    away_team_data = get_team_expenditure_data(away_team_id, season)

    # Update the matches_df with the expenditure data
    for key, value in home_team_data.items():
        matches_df.at[row.Index, f'home_team_{key.lower()}'] = value
    for key, value in away_team_data.items():
        matches_df.at[row.Index, f'away_team_{key.lower()}'] = value

# Display the updated matches_df
matches_df.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA,home_team_expenditure,away_team_expenditure
0,1729,1729,1729,2008/2009,1,2008-08-17,489042,10260,10261,1,...,5.5,12.00,1.30,4.75,10.0,1.29,4.50,11.00,45250000.0,35480000.0
1,1730,1729,1729,2008/2009,1,2008-08-16,489043,9825,8659,1,...,6.0,13.00,1.22,5.50,13.0,1.22,5.00,13.00,40150000.0,24320000.0
2,1731,1729,1729,2008/2009,1,2008-08-16,489044,8472,8650,0,...,3.8,1.65,5.00,3.40,1.7,4.50,3.40,1.73,33110000.0,73450000.0
3,1732,1729,1729,2008/2009,1,2008-08-16,489045,8654,8528,2,...,3.5,4.35,1.91,3.25,4.0,1.91,3.25,3.80,16000000.0,24100000.0
4,1733,1729,1729,2008/2009,1,2008-08-17,489046,10252,8456,4,...,3.5,4.35,1.91,3.25,4.0,1.91,3.30,3.75,55900000.0,157350000.0


# Merge matches_df and team_attributes_df

## Preliminary EDA and Cleaning on team_attributes_df

In [30]:
# Print info of team_attributes_df
team_attributes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458 entries, 0 to 1457
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              1458 non-null   int64         
 1   team_fifa_api_id                1458 non-null   int64         
 2   team_api_id                     1458 non-null   int64         
 3   date                            1458 non-null   datetime64[ns]
 4   buildUpPlaySpeed                1458 non-null   int64         
 5   buildUpPlaySpeedClass           1458 non-null   object        
 6   buildUpPlayDribbling            489 non-null    float64       
 7   buildUpPlayDribblingClass       1458 non-null   object        
 8   buildUpPlayPassing              1458 non-null   int64         
 9   buildUpPlayPassingClass         1458 non-null   object        
 10  buildUpPlayPositioningClass     1458 non-null   object        
 11  chan

In [31]:
# Identify categorical columns in team_attributes_df
categorical_columns = team_attributes_df.select_dtypes(include=['object']).columns

# Print unique values for each categorical column
for col in categorical_columns:
    print(f"Unique values in '{col}':")
    print(team_attributes_df[col].unique())

Unique values in 'buildUpPlaySpeedClass':
['Balanced' 'Fast' 'Slow']
Unique values in 'buildUpPlayDribblingClass':
['Little' 'Normal' 'Lots']
Unique values in 'buildUpPlayPassingClass':
['Mixed' 'Long' 'Short']
Unique values in 'buildUpPlayPositioningClass':
['Organised' 'Free Form']
Unique values in 'chanceCreationPassingClass':
['Normal' 'Risky' 'Safe']
Unique values in 'chanceCreationCrossingClass':
['Normal' 'Lots' 'Little']
Unique values in 'chanceCreationShootingClass':
['Normal' 'Lots' 'Little']
Unique values in 'chanceCreationPositioningClass':
['Organised' 'Free Form']
Unique values in 'defencePressureClass':
['Medium' 'Deep' 'High']
Unique values in 'defenceAggressionClass':
['Press' 'Double' 'Contain']
Unique values in 'defenceTeamWidthClass':
['Normal' 'Wide' 'Narrow']
Unique values in 'defenceDefenderLineClass':
['Cover' 'Offside Trap']


In [32]:
# Count NaNs in each column
nan_counts = team_attributes_df.isna().sum()

# Filter and display columns with NaNs greater than 0
columns_with_nans = nan_counts[nan_counts > 0]
print("Columns with NaN values in team_attributes_df:")
print(columns_with_nans)

Columns with NaN values in team_attributes_df:
buildUpPlayDribbling    969
dtype: int64


In [33]:
# Remove buildUpPlayDribbling column
team_attributes_df.drop(columns = ["buildUpPlayDribbling"], inplace = True)

## Merge

In [35]:
# Create the 'season' column based on the month of each date
team_attributes_df['season'] = team_attributes_df['date'].apply(
    lambda x: f"{x.year}/{x.year + 1}" if x.month > 8 else f"{x.year - 1}/{x.year}"
)

# Drop unnecessary columns from Team_Attributes_df
team_attributes_to_merge = team_attributes_df.drop(columns=["id", "team_fifa_api_id", "date"])

# Add prefixes to differentiate home and away attributes
home_team_attributes = team_attributes_to_merge.add_prefix("home_")
away_team_attributes = team_attributes_to_merge.add_prefix("away_")

# Merge home team attributes based on home_team_api_id and season
matches_df = matches_df.merge(
    home_team_attributes,
    left_on=["home_team_api_id", "season"],
    right_on=["home_team_api_id", "home_season"],
    how="left"
)

# Merge away team attributes based on away_team_api_id and season
matches_df = matches_df.merge(
    away_team_attributes,
    left_on=["away_team_api_id", "season"],
    right_on=["away_team_api_id", "away_season"],
    how="left"
)

# Drop the 'home_season' and 'away_season' columns used for merging
matches_df = matches_df.drop(columns=["home_season", "away_season"])

# Display the updated matches_df
matches_df.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,away_chanceCreationShooting,away_chanceCreationShootingClass,away_chanceCreationPositioningClass,away_defencePressure,away_defencePressureClass,away_defenceAggression,away_defenceAggressionClass,away_defenceTeamWidth,away_defenceTeamWidthClass,away_defenceDefenderLineClass
0,1729,1729,1729,2008/2009,1,2008-08-17,489042,10260,10261,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1730,1729,1729,2008/2009,1,2008-08-16,489043,9825,8659,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1731,1729,1729,2008/2009,1,2008-08-16,489044,8472,8650,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1732,1729,1729,2008/2009,1,2008-08-16,489045,8654,8528,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1733,1729,1729,2008/2009,1,2008-08-17,489046,10252,8456,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Merge matches_df and player_attributes_df

## Preliminary EDA and Cleaning on player_attributes_df

In [38]:
# Print info of player_attributes_df
player_attributes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183978 entries, 0 to 183977
Data columns (total 42 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   id                   183978 non-null  int64         
 1   player_fifa_api_id   183978 non-null  int64         
 2   player_api_id        183978 non-null  int64         
 3   date                 183978 non-null  datetime64[ns]
 4   overall_rating       183142 non-null  float64       
 5   potential            183142 non-null  float64       
 6   preferred_foot       183142 non-null  object        
 7   attacking_work_rate  180748 non-null  object        
 8   defensive_work_rate  183142 non-null  object        
 9   crossing             183142 non-null  float64       
 10  finishing            183142 non-null  float64       
 11  heading_accuracy     183142 non-null  float64       
 12  short_passing        183142 non-null  float64       
 13  volleys       

In [39]:
# Identify categorical columns in player_attributes_df
categorical_columns = player_attributes_df.select_dtypes(include=['object']).columns

# Print unique values for each categorical column
for col in categorical_columns:
    print(f"Unique values in '{col}':")
    print(player_attributes_df[col].unique())

Unique values in 'preferred_foot':
['right' 'left' None]
Unique values in 'attacking_work_rate':
['medium' 'high' None 'low' 'None' 'le' 'norm' 'stoc' 'y']
Unique values in 'defensive_work_rate':
['medium' 'high' 'low' '_0' None '5' 'ean' 'o' '1' 'ormal' '7' '2' '8' '4'
 'tocky' '0' '3' '6' '9' 'es']


In [40]:
# Count NaNs in each column
nan_counts = player_attributes_df.isna().sum()

# Filter and display columns with NaNs greater than 0
columns_with_nans = nan_counts[nan_counts > 0]
print("Columns with NaN values in team_attributes_df:")
print(columns_with_nans)

Columns with NaN values in team_attributes_df:
overall_rating          836
potential               836
preferred_foot          836
attacking_work_rate    3230
defensive_work_rate     836
crossing                836
finishing               836
heading_accuracy        836
short_passing           836
volleys                2713
dribbling               836
curve                  2713
free_kick_accuracy      836
long_passing            836
ball_control            836
acceleration            836
sprint_speed            836
agility                2713
reactions               836
balance                2713
shot_power              836
jumping                2713
stamina                 836
strength                836
long_shots              836
aggression              836
interceptions           836
positioning             836
vision                 2713
penalties               836
marking                 836
standing_tackle         836
sliding_tackle         2713
gk_diving               836
g

In [41]:
# There are some unreasonable values in the 'attacking_work_rate' and 'defensive_work_rate'

# List of valid values for attacking and defensive work rates
valid_work_rates = ['medium', 'high', 'low']

# Function to clean work rate columns
def clean_work_rate_columns(df, columns, valid_values):
    for column in columns:
        # Replace invalid values with NaN
        df[column] = df[column].apply(lambda x: x if x in valid_values else np.nan)
    return df

columns_to_clean = ['attacking_work_rate', 'defensive_work_rate']

# Clean the DataFrame
player_attributes_df = clean_work_rate_columns(player_attributes_df, columns_to_clean, valid_work_rates)

# Check results
player_attributes_df[columns_to_clean].head()

,attacking_work_rate,defensive_work_rate
0,medium,medium
1,medium,medium
2,medium,medium
3,medium,medium
4,medium,medium


## Merge

In [43]:
def process_player_column(player_col):
    '''
    A function for processing each player column
    '''
    temp_data = []

    for _, match in matches_df.iterrows():
        match_date = match['date']
        player_id = match[player_col]

        # Filter attributes for the specific player
        player_attributes = player_attributes_df[player_attributes_df['player_api_id'] == player_id].copy()

        # If attributes for the player exist, find the closest date
        if not player_attributes.empty:
            player_attributes['date_diff'] = (player_attributes['date'] - match_date).abs()
            closest_row = player_attributes.loc[player_attributes['date_diff'].idxmin(), attributes]
            closest_row = closest_row.add_prefix(f"{player_col}_")
            closest_row['id'] = match['id']  # Match on 'id' for merging
            temp_data.append(closest_row)

    # Convert collected data for this player column to a DataFrame
    player_data_df = pd.DataFrame(temp_data)
    return player_data_df

# List of player columns in matches_df
player_columns = [f"home_player_{i}" for i in range(1, 12)] + [f"away_player_{i}" for i in range(1, 12)]

# Define player attributes to keep
attributes = [
    'overall_rating', 'potential', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate',
    'crossing', 'finishing', 'heading_accuracy', 'short_passing', 'volleys', 'dribbling', 'curve',
    'free_kick_accuracy', 'long_passing', 'ball_control', 'acceleration', 'sprint_speed', 'agility',
    'reactions', 'balance', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots', 'aggression',
    'interceptions', 'positioning', 'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
    'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes'
]

# Initialize a list to collect processed data for each player
all_player_data = []

# parallelized processing
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = {executor.submit(process_player_column, player_col): player_col for player_col in player_columns}

    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="Processing player columns"):
        player_data_df = future.result()
        all_player_data.append(player_data_df)

# Merge all player data back into matches_df
for player_data_df in all_player_data:
    matches_df = matches_df.merge(player_data_df, how='left', on='id')

# Display the updated matches_df
matches_df.head()

Processing player columns: 100%|████████████████| 22/22 [04:56<00:00, 13.49s/it]


,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,away_player_11_vision,away_player_11_penalties,away_player_11_marking,away_player_11_standing_tackle,away_player_11_sliding_tackle,away_player_11_gk_diving,away_player_11_gk_handling,away_player_11_gk_kicking,away_player_11_gk_positioning,away_player_11_gk_reflexes
0,1729,1729,1729,2008/2009,1,2008-08-17,489042,10260,10261,1,...,70.0,70.0,26.0,30.0,20.0,12.0,22.0,48.0,22.0,22.0
1,1730,1729,1729,2008/2009,1,2008-08-16,489043,9825,8659,1,...,54.0,57.0,22.0,31.0,20.0,10.0,22.0,35.0,22.0,22.0
2,1731,1729,1729,2008/2009,1,2008-08-16,489044,8472,8650,0,...,81.0,89.0,21.0,20.0,29.0,10.0,20.0,51.0,20.0,20.0
3,1732,1729,1729,2008/2009,1,2008-08-16,489045,8654,8528,2,...,81.0,65.0,60.0,45.0,52.0,9.0,21.0,47.0,21.0,21.0
4,1733,1729,1729,2008/2009,1,2008-08-17,489046,10252,8456,4,...,64.0,53.0,43.0,50.0,46.0,1.0,21.0,51.0,21.0,21.0


# Calculate Match Outcome (Target Feature)

In [45]:
def calculate_match_result(matches_df):
    """
    Calculate the result of each match and add it as a new column.

    Parameters:
    matches_df (pd.DataFrame): DataFrame containing match data with 'home_team_goal' and 'away_team_goal' columns.

    Returns:
    pd.DataFrame: DataFrame with an additional 'match_result' column indicating 'Home Win', 'Draw', or 'Away Win'.
    """
    # Make a copy of the DataFrame
    matches_df = matches_df.copy()

    # Define the result function to calculate the match outcome
    def result(row):
        if row['home_team_goal'] > row['away_team_goal']:
            return 'Home Win'
        elif row['home_team_goal'] < row['away_team_goal']:
            return 'Away Win'
        else:
            return 'Draw'

    # Apply this function to each row of the dataframe and create the 'match_result' column
    matches_df['match_result'] = matches_df.apply(result, axis=1)

    return matches_df

# Applying calculate_match_result function to the match_df DataFrame
matches_df = calculate_match_result(matches_df)
print(matches_df[['home_team_goal', 'away_team_goal', 'match_result']].head())

   home_team_goal  away_team_goal match_result
0               1               1         Draw
1               1               0     Home Win
2               0               1     Away Win
3               2               1     Home Win
4               4               2     Home Win


# Save to CSV

In [48]:
# Create the directory
output_dir = "Data"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the matches_df DataFrame to a CSV file
output_path = os.path.join(output_dir, "matches_1.csv")
matches_df.to_csv(output_path, index=False)

print(f"matches_df has been saved to {output_path}")

matches_df has been saved to Data/matches_1.csv
